In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck18
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Load your galaxy data (cartesian coordinates and redshifts)
df = pd.read_csv('../data/galaxies_cartesian.csv')

# Function to apply redshift-space distortions (RSD) by adding LOS velocity dispersion
def apply_rsd(df, velocity_dispersion_kms):
    c = 3e5  # speed of light km/s
    df_rsd = df.copy()

    # Random velocity along line of sight (z-direction)
    sigma_v = velocity_dispersion_kms  # km/s
    delta_v = np.random.normal(0, sigma_v, size=len(df_rsd))

    # Convert velocity shift to distance shift along line of sight (z)
    delta_r = (delta_v / c) * df_rsd['r']  # r is the comoving distance in Mpc

    # Add distortion along line of sight (z-axis)
    df_rsd['z_cart'] += delta_r

    return df_rsd

# Function to compute 2-point correlation function
def count_pairs(data1, data2, bins):
    tree1 = cKDTree(data1)
    counts = np.zeros(len(bins) - 1)
    for i in range(len(bins) - 1):
        counts[i] = tree1.count_neighbors(cKDTree(data2), bins[i+1]) - tree1.count_neighbors(cKDTree(data2), bins[i])
    return counts

def compute_correlation(df_subset, bins):
    galaxy_positions = df_subset[['x', 'y', 'z_cart']].values
    # Create random catalog within bounds of data subset
    x_min, x_max = df_subset['x'].min(), df_subset['x'].max()
    y_min, y_max = df_subset['y'].min(), df_subset['y'].max()
    z_min, z_max = df_subset['z_cart'].min(), df_subset['z_cart'].max()
    num_random = 5 * len(df_subset)
    random_positions = np.column_stack((
        np.random.uniform(x_min, x_max, num_random),
        np.random.uniform(y_min, y_max, num_random),
        np.random.uniform(z_min, z_max, num_random)
    ))

    DD = count_pairs(galaxy_positions, galaxy_positions, bins)
    RR = count_pairs(random_positions, random_positions, bins)
    DR = count_pairs(galaxy_positions, random_positions, bins)

    DD = DD / (len(galaxy_positions) * (len(galaxy_positions) - 1) / 2)
    RR = RR / (len(random_positions) * (len(random_positions) - 1) / 2)
    DR = DR / (len(galaxy_positions) * len(random_positions))

    xi = (DD - 2 * DR + RR) / RR
    r = 0.5 * (bins[:-1] + bins[1:])
    return r, xi

# Prepare initial bins
bins = np.linspace(0, 20, 20)

vel_input = widgets.FloatText(value=300, description='Velocity Dispersion (km/s):')
z_min_input = widgets.FloatText(value=0.01, description='z min:')
z_max_input = widgets.FloatText(value=0.3, description='z max:')
run_button = widgets.Button(description='Update Plot')

output = widgets.Output()

In [3]:
def update_plots(change=None):
    with output:
        clear_output(wait=True)

        # Get values from text boxes
        velocity_dispersion = vel_input.value
        zmin = z_min_input.value
        zmax = z_max_input.value

        # Filter and check
        df_filtered = df[(df['z'] >= zmin) & (df['z'] <= zmax)].copy()
        if df_filtered.empty:
            print("No galaxies in the selected redshift range.")
            return

        df_filtered['r'] = Planck18.comoving_distance(df_filtered['z']).value
        df_rsd = apply_rsd(df_filtered, velocity_dispersion)

        r_orig, xi_orig = compute_correlation(df_filtered, bins)
        r_rsd, xi_rsd = compute_correlation(df_rsd, bins)

        x_min = min(df_filtered['x'].min(), df_rsd['x'].min())
        x_max = max(df_filtered['x'].max(), df_rsd['x'].max())
        y_min = min(df_filtered['y'].min(), df_rsd['y'].min())
        y_max = max(df_filtered['y'].max(), df_rsd['y'].max())
        z_min = min(df_filtered['z_cart'].min(), df_rsd['z_cart'].min())
        z_max = max(df_filtered['z_cart'].max(), df_rsd['z_cart'].max())

        fig = plt.figure(figsize=(12, 8))

        ax1 = fig.add_subplot(221, projection='3d')
        ax1.scatter(df_filtered['x'], df_filtered['y'], df_filtered['z_cart'], s=2, alpha=0.5, color='blue')
        ax1.set_title('Original 3D Galaxy Distribution')
        ax1.set_xlabel('X (Mpc)')
        ax1.set_ylabel('Y (Mpc)')
        ax1.set_zlabel('Z (Mpc)')
        ax1.set_xlim(x_min, x_max)
        ax1.set_ylim(y_min, y_max)
        ax1.set_zlim(z_min, z_max)

        ax3 = fig.add_subplot(222, projection='3d')
        ax3.scatter(df_rsd['x'], df_rsd['y'], df_rsd['z_cart'], s=2, alpha=0.5, color='red')
        ax3.set_title(f'RSD 3D Galaxy Distribution\nVelocity Dispersion = {velocity_dispersion} km/s')
        ax3.set_xlabel('X (Mpc)')
        ax3.set_ylabel('Y (Mpc)')
        ax3.set_zlabel('Z (Mpc)')
        ax3.set_xlim(x_min, x_max)
        ax3.set_ylim(y_min, y_max)
        ax3.set_zlim(z_min, z_max)

        ax2 = fig.add_subplot(212)
        ax2.plot(r_orig, xi_orig, marker='o', linestyle='-', color='blue', label='Original')
        ax2.plot(r_rsd, xi_rsd, marker='o', linestyle='-', color='red', label='With RSD')
        ax2.axhline(0, color='gray', linestyle='--')
        ax2.set_xlabel('Separation r (Mpc)')
        ax2.set_ylabel('Correlation function ξ(r)')
        ax2.set_title('Two-Point Correlation Function')
        ax2.grid(True)
        ax2.legend()

        plt.tight_layout()
        plt.show()

In [ ]:
run_button.on_click(update_plots)
display(vel_input, z_min_input, z_max_input, run_button, output)